<img src="https://web.unipv.it/wp-content/uploads/2021/04/MAIN_LOGO_suduerighe_1000x1000_Granata-su-bianco-300x118.png" style="float: right; width: 20em; margin-left: 100px;">

# Ornamental Value Estimation

## No-SQL Database for an Interdisciplinary Study on Autoctone Botanic Species

### Course
Digital Content Retrieval ([CSU](http://csu.unipv.it/didattica/))

### Team Members
Aiman Al Masoud, Amato Francesco, Andrei Blindu, Mattia Bottini, Davide Lotito, Francesco Marinelli, Daniele Murer, Riccardo Petri, Fabio Tagliani

### Academic year
2021/22

![License](https://img.shields.io/github/license/Ileriayo/markdown-badges?style=for-the-badge)

## Introduction 

The raw dataset, in `.xls` format, was obtained from the CSU website:

[http://csu.unipv.it/wp-content/uploads/2022/01/Database_ornamentali_grezzo.xls](http://csu.unipv.it/wp-content/uploads/2022/01/Database_ornamentali_grezzo.xls)

### Data manipulation steps

1.  The raw data was loaded into a pandas DataFrame.

2.  The data was cleaned up, and the values for the following data processing steps were made consistent with each other. 

3.  Then the data was processed to obtain the "ornamental value" metric, using 8 relevant parameters to calculate it. The function that computes this metric, toghether with the data, is based on the work by F. Colombini available at [http://csu.unipv.it/wp-content/uploads/2022/01/Tesi-Colombini.pdf](http://csu.unipv.it/wp-content/uploads/2022/01/Tesi-Colombini.pdf).

    The **ornamental value** is a figure of merit that can be used in order to decide if a specific species can be planted in a given area. The higher the ornamental value, the better the final result should be.

4.  Then the data was converted into a list of `json` objects and stored into a remote MongoDB instance.

MongoDB was found to be ideal for this kind of application, since different plant species have different characteristics, and MongoDB allows for a loose structuring of the documents as opposed to relational databases.

We used the free online MongoDB hosting service [Atlas](https://www.mongodb.com/atlas/database) to store the data, so that we could all collaborate on the project. We used the `pymongo` Python API to interact with the remote instance of the database.

If you wish to download the dataset in `json` format use this download link: <a href="https://drive.google.com/uc?export=download&id=1qbFqC_eSVcBgorBP0HvW2fprnrDihGQP" title="Download the dataset" target="_blank">dataset</a>.

## Data Import

In [1]:
# Import the libraries
import pandas as pd
import json
import pymongo

from utilities import *

# Import the dataset
raw = 'http://csu.unipv.it/wp-content/uploads/2022/01/Database_ornamentali_grezzo.xls'

### Converting the `.xls` to a DataFrame

In [2]:
# Create the DataFrame starting from the given path
raw_df = pd.read_excel(raw)

In [3]:
raw_df

,TAXON,4030,2330+6210(acid),6210(calc),FORMA BIOLOGICA,DIMENSIONE FIORE,VISTOSITA',PERIODO FIORITURA,DURATA,PRESENZA MERCATO,MERCATO,AROMATICA,IMPOLLINAZIONE,ALLERGENICITA’,TOSSICITA’
0,Aegilops_geniculata,NaN,NaN,X,1.0,NaN,0,Maggio-giugno,NaN,NaN,NaN,0,0,1,1
1,Achillea_millefolium,X,X,NaN,2.0,"2,5 - 3 x 4- 5 cm",1,Maggio-settembre,NaN,Achillea millefolium,NaN,1,1,1,1
2,Achillea_tomentosa,NaN,X,X,2.0,Capolini 5-7 mm,1,Giugno-agosto,NaN,Achillea millefolium,NaN,1,1,1,1
3,Agrostis_capillaris,X,X,NaN,2.0,NaN,0,Giugno-agosto,NaN,Agrostis capillaris,NaN,0,0,1,1
4,Aira_caryophyllea,X,X,X,1.0,NaN,0,Aprile-maggio,NaN,NaN,NaN,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Vincetoxicum_hirundinaria,X,NaN,NaN,2.0,5-6 mm,1,Maggio-agoso,NaN,NaN,NaN,0,1,1,0
218,Viola_arvensis,NaN,X,X,1.0,"1-1,5 mm",1,Febbraio-settembre,NaN,Viola cornuta,NaN,1,1,1,1
219,Viola_canina,X,NaN,NaN,2.0,"1,5-2,5 mm",1,Aprile-luglio,NaN,Viola tricolor,NaN,1,1,1,1
220,Vulpia_ciliata,NaN,X,X,1.0,(30-40 mm Ø) fiori(20-25 mm),0,Aprile-giugno,NaN,Vulpia ciliata,NaN,0,0,1,1


## Compute the Ornamental Score of the Plants

### Test: Compute the Ornamental Value for a Single Plant

Note that the `get_score_sum` and `get_ornamental_index_value` functions are contained into the other file named `utilities.py`.

In [4]:
print('Score sum: ' + str(get_score_sum(raw_df.loc[0])))
print('Ornamental value: ' + str(get_ornamental_index_value(raw_df.loc[0])))

Score sum: 4.0
Ornamental value: 1


### Goal: Repeat the Previous Step for All the Plants

In [5]:
# Score sum
score_sum_series = raw_df.apply(get_score_sum, axis=1)
score_sum_series

0       4.0
1      11.0
2      10.0
3       7.0
4       3.0
       ... 
217     7.0
218    10.0
219    10.0
220     6.0
221     6.0
Length: 222, dtype: float64

In [6]:
# Ornamental value
ornamental_value_series = raw_df.apply(get_ornamental_index_value, axis=1)
ornamental_value_series

0      1.0
1      3.0
2      3.0
3      2.0
4      1.0
      ... 
217    2.0
218    3.0
219    3.0
220    2.0
221    2.0
Length: 222, dtype: float64

## Generate the `json` Object

We want to add our figures of merit to each plant and then create a single `json` file containing all this information.

In [7]:
# Add the figures of merit to each plant
raw_df['SOMMA DEI PUNTEGGI'] = score_sum_series
raw_df['INDICE DI ORNAMENTALITA\''] = ornamental_value_series
raw_df

,TAXON,4030,2330+6210(acid),6210(calc),FORMA BIOLOGICA,DIMENSIONE FIORE,VISTOSITA',PERIODO FIORITURA,DURATA,PRESENZA MERCATO,MERCATO,AROMATICA,IMPOLLINAZIONE,ALLERGENICITA’,TOSSICITA’,SOMMA DEI PUNTEGGI,INDICE DI ORNAMENTALITA'
0,Aegilops_geniculata,NaN,NaN,X,1.0,NaN,0,Maggio-giugno,NaN,NaN,NaN,0,0,1,1,4.0,1.0
1,Achillea_millefolium,X,X,NaN,2.0,"2,5 - 3 x 4- 5 cm",1,Maggio-settembre,NaN,Achillea millefolium,NaN,1,1,1,1,11.0,3.0
2,Achillea_tomentosa,NaN,X,X,2.0,Capolini 5-7 mm,1,Giugno-agosto,NaN,Achillea millefolium,NaN,1,1,1,1,10.0,3.0
3,Agrostis_capillaris,X,X,NaN,2.0,NaN,0,Giugno-agosto,NaN,Agrostis capillaris,NaN,0,0,1,1,7.0,2.0
4,Aira_caryophyllea,X,X,X,1.0,NaN,0,Aprile-maggio,NaN,NaN,NaN,0,0,0,1,3.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,Vincetoxicum_hirundinaria,X,NaN,NaN,2.0,5-6 mm,1,Maggio-agoso,NaN,NaN,NaN,0,1,1,0,7.0,2.0
218,Viola_arvensis,NaN,X,X,1.0,"1-1,5 mm",1,Febbraio-settembre,NaN,Viola cornuta,NaN,1,1,1,1,10.0,3.0
219,Viola_canina,X,NaN,NaN,2.0,"1,5-2,5 mm",1,Aprile-luglio,NaN,Viola tricolor,NaN,1,1,1,1,10.0,3.0
220,Vulpia_ciliata,NaN,X,X,1.0,(30-40 mm Ø) fiori(20-25 mm),0,Aprile-giugno,NaN,Vulpia ciliata,NaN,0,0,1,1,6.0,2.0


Note that the `dataframe_to_json_array` function is contained into the other file named `utilities.py`.

In [8]:
# Convert the DataFrame to a json object
json_object = json.loads(raw_df.to_json(orient='records'))
json_object

[{'TAXON': 'Aegilops_geniculata',
  '4030': None,
  '2330+6210(acid)': None,
  '6210(calc)': 'X',
  'FORMA BIOLOGICA': 1.0,
  'DIMENSIONE FIORE': None,
  "VISTOSITA'": 0,
  'PERIODO FIORITURA': 'Maggio-giugno',
  'DURATA': None,
  'PRESENZA MERCATO': None,
  'MERCATO': None,
  'AROMATICA': 0,
  'IMPOLLINAZIONE': 0,
  'ALLERGENICITA’': 1,
  'TOSSICITA’': 1,
  'SOMMA DEI PUNTEGGI': 4.0,
  "INDICE DI ORNAMENTALITA'": 1.0},
 {'TAXON': 'Achillea_millefolium',
  '4030': 'X',
  '2330+6210(acid)': 'X',
  '6210(calc)': None,
  'FORMA BIOLOGICA': 2.0,
  'DIMENSIONE FIORE': ' 2,5 - 3 x 4- 5 cm ',
  "VISTOSITA'": 1,
  'PERIODO FIORITURA': 'Maggio-settembre',
  'DURATA': None,
  'PRESENZA MERCATO': 'Achillea millefolium',
  'MERCATO': None,
  'AROMATICA': 1,
  'IMPOLLINAZIONE': 1,
  'ALLERGENICITA’': 1,
  'TOSSICITA’': 1,
  'SOMMA DEI PUNTEGGI': 11.0,
  "INDICE DI ORNAMENTALITA'": 3.0},
 {'TAXON': 'Achillea_tomentosa',
  '4030': None,
  '2330+6210(acid)': 'X',
  '6210(calc)': 'X',
  'FORMA BIOLOGIC

## Store the `json` Object

### Locally

Note that the `store_locally` function is contained into the other file named `utilities.py`.

The file will be saved in the root of this repository.

In [9]:
store_locally(json_object)

### Into Our MongoDB Istance

Note that the `store_into_mongodb_istance` function is contained into the other file named `utilities.py`.

In [10]:
store_into_mongodb_istance(json_object)

## Test: Load a Single Document (Plant) From the MongoDB Instance

In [11]:
client = pymongo.MongoClient(get_connection_string())

# Test: show the first record
client.OVE.data.find()[0]

{'_id': ObjectId('622375de530de6ccfba98f9d'),
 'TAXON': 'Aegilops_geniculata',
 '4030': None,
 '2330+6210(acid)': None,
 '6210(calc)': 'X',
 'FORMA BIOLOGICA': 1.0,
 'DIMENSIONE FIORE': None,
 "VISTOSITA'": 0,
 'PERIODO FIORITURA': 'Maggio-giugno',
 'DURATA': None,
 'PRESENZA MERCATO': None,
 'MERCATO': None,
 'AROMATICA': 0,
 'IMPOLLINAZIONE': 0,
 'ALLERGENICITA’': 1,
 'TOSSICITA’': 1,
 'SOMMA DEI PUNTEGGI': 4.0,
 "INDICE DI ORNAMENTALITA'": 1.0}